In [1]:
# Generate the video for the env with fake action

In [2]:
import sys
sys.path.insert(0, "torchdrivesim")

In [3]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

from gym_env import SingleAgentWrapper, WaypointSuiteEnvConfig, WaypointSuiteEnv
from env_utils import load_waypoint_suite_env_config

Commercial access denied and fallback to check for academic access.


In [4]:
# model = PPO.load("models/PPO_1700104602/model")
model = SAC.load("smooth_model")

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 73.74GB > 23.04GB
  warnings.warn(


In [5]:
env_config = load_waypoint_suite_env_config("env_configs/new_new_waypoint_suite_env_config.yml")

In [6]:
def make_env():
    gym.register('torchdrivesim/waypoint-suite-v0', entry_point=lambda args: SingleAgentWrapper(WaypointSuiteEnv(cfg=args)))
    env = gym.make('torchdrivesim/waypoint-suite-v0', args=env_config)
    env = Monitor(env)  
    return env

In [7]:
def main(cfg, model, annotated_mesh=None):
    env = DummyVecEnv([make_env])
    # env = SubprocVecEnv([make_env] * 3)
    env = VecFrameStack(env, n_stack=3, channels_order="first")
    obs = np.expand_dims(env.reset()[0], 0)
    states = None
    episode_starts = [True]

    if annotated_mesh is not None:
        innermost_simulator = env.simulator.get_innermost_simulator()
        innermost_simulator.renderer.add_static_meshes(annotated_mesh)

    for i in range(cfg.iai_gym.steps):
        actions, states = model.predict(
         obs,  
         state=states,
         episode_start=episode_starts,
         deterministic=True,
        )
        # actions = [actions]
        # print("actions")
        # print(actions)
        # print("obs")
        # print(obs)
        obs, reward, done, info = env.step(actions)            
        print(f"reward at step{i}", reward)
        # if done:
        #     break
    env.close()

In [8]:
env_config.iai_gym.render_mode = "video"
env_config.iai_gym.steps = 100

In [9]:
env_config

WaypointSuiteEnvConfig(iai_gym=IAIGymEnvConfig(simulator=TorchDriveConfig(renderer=RendererConfig(backend='default', render_agent_direction=True, left_handed_coordinates=True, highlight_ego_vehicle=True), single_agent_rendering=False, collision_metric=<CollisionMetric.discs: 'discs'>, offroad_threshold=0.5, left_handed_coordinates=True), driving_surface_mesh_path='../resources/maps/carla/meshes/Town01_driving_surface_mesh.pkl', render_mode='video', video_filename='rendered_video.mp4', location='Town01', res=1024, fov=450.0, center=[0.0, 0.0], left_handed=True, agent_count=5, ego_state=None, use_mock_lights=True, use_area_initialize=False, max_environment_steps=200, use_background_traffic=True, terminated_at_infraction=True, ego_only=False, seed=None), locations=['Town07', 'Town07', 'Town03', 'Town03', 'Town01'], waypointsuite=[[[-88.29564666748047, -118.39412689208984], [-96.58080291748047, -104.98006439208984], [-109.99486541748047, -101.42928314208984], [-128.53783416748047, -101.823

In [10]:
main(env_config, model)

INFO:gym_env:seed: 3848149754
INFO:gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.5 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-0.05) if (self.last_psi is not None) else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
        return r



waypoints shape
torch.Size([1, 31, 11, 1, 2])
1
waypoints_mesh
BirdviewMesh(verts=tensor([[[196.5526,  55.1527],
         [198.5526,  55.1527],
         [198.1706,  56.3283],
         [197.1706,  57.0548],
         [195.9346,  57.0548],
         [194.9346,  56.3283],
         [194.5526,  55.1527],
         [194.9345,  53.9772],
         [195.9346,  53.2506],
         [197.1706,  53.2506],
         [198.1706,  53.9772]]], device='cuda:0'), faces=tensor([[[ 0,  1,  2],
         [ 0,  2,  3],
         [ 0,  3,  4],
         [ 0,  4,  5],
         [ 0,  5,  6],
         [ 0,  6,  7],
         [ 0,  7,  8],
         [ 0,  8,  9],
         [ 0,  9, 10],
         [ 0, 10,  1]]], device='cuda:0'), _verts_fill=0.0, _faces_fill=0, categories=['goal_waypoint'], colors={}, zs={}, vert_category=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), _cat_fill=0)
1
waypoints_mesh
BirdviewMesh(verts=tensor([[[196.5526,  55.1527],
         [198.5526,  55.1527],
         [198.1706,  56.3283],
   

/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be uint8, actual type: float32
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


1
waypoints_mesh
BirdviewMesh(verts=tensor([[[169.7121,  54.6900],
         [171.7121,  54.6900],
         [171.3301,  55.8655],
         [170.3301,  56.5921],
         [169.0940,  56.5921],
         [168.0940,  55.8655],
         [167.7121,  54.6900],
         [168.0940,  53.5144],
         [169.0940,  52.7878],
         [170.3301,  52.7878],
         [171.3301,  53.5144]]], device='cuda:0'), faces=tensor([[[ 0,  1,  2],
         [ 0,  2,  3],
         [ 0,  3,  4],
         [ 0,  4,  5],
         [ 0,  5,  6],
         [ 0,  6,  7],
         [ 0,  7,  8],
         [ 0,  8,  9],
         [ 0,  9, 10],
         [ 0, 10,  1]]], device='cuda:0'), _verts_fill=0.0, _faces_fill=0, categories=['goal_waypoint'], colors={}, zs={}, vert_category=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), _cat_fill=0)
1
waypoints_mesh
BirdviewMesh(verts=tensor([[[169.7121,  54.6900],
         [171.7121,  54.6900],
         [171.3301,  55.8655],
         [170.3301,  56.5921],
         [169.0940

/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:228: UserWarning: WARN: Expects `terminated` signal to be a boolean, actual type: <class 'torch.Tensor'>
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be uint8, actual type: float32
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/usr/local/lib/python3.8/dist-packages/gymnasium/utils/passive_env_checker.py:246: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'torch.Tensor'>
  logger.warn(


1
waypoints_mesh
BirdviewMesh(verts=tensor([[[169.7121,  54.6900],
         [171.7121,  54.6900],
         [171.3301,  55.8655],
         [170.3301,  56.5921],
         [169.0940,  56.5921],
         [168.0940,  55.8655],
         [167.7121,  54.6900],
         [168.0940,  53.5144],
         [169.0940,  52.7878],
         [170.3301,  52.7878],
         [171.3301,  53.5144]]], device='cuda:0'), faces=tensor([[[ 0,  1,  2],
         [ 0,  2,  3],
         [ 0,  3,  4],
         [ 0,  4,  5],
         [ 0,  5,  6],
         [ 0,  6,  7],
         [ 0,  7,  8],
         [ 0,  8,  9],
         [ 0,  9, 10],
         [ 0, 10,  1]]], device='cuda:0'), _verts_fill=0.0, _faces_fill=0, categories=['goal_waypoint'], colors={}, zs={}, vert_category=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), _cat_fill=0)
1
waypoints_mesh
BirdviewMesh(verts=tensor([[[169.7121,  54.6900],
         [171.7121,  54.6900],
         [171.3301,  55.8655],
         [170.3301,  56.5921],
         [169.0940